In [ ]:
import pygame
import random
import sys

# Initialize Pygame
pygame.init()

# Set up display
width, height = 840, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("El Examen")

# Colors
white = (255, 255, 255)
black = (0, 0, 0)

#Font
font = pygame.font.Font(None,32)
font_2 = pygame.font.Font(None,25)

#Buttons
button_width, button_height = 120, 50

button_color = black
button_math = pygame.Rect(120, 150, button_width, button_height)
button_chem = pygame.Rect(360, 150, button_width, button_height)
button_eng = pygame.Rect(600, 150, button_width, button_height)

# Create a clock object to control the frame rate
clock = pygame.time.Clock()
fps = 60  # Set the desired frames per second

running = True
Math_Game_1 = Math_Game_2 = Math_Game_3 = Eng_Game_1 = Eng_Game_2 = Eng_Game_3 = Chem_Game_1 = Chem_Game_2 = Chem_Game_3 = False

i=0

def button_dynamics(button, name, subname_1, subname_2, subname_3):
        hover = button.collidepoint(mouse_x, mouse_y)

        if hover and button.height < 345:
            button.height += 5
        elif not hover and button.height > 50:
            button.height -= 5
        
        button = pygame.Rect(button.x, button.y, button.width, button.height)
        pygame.draw.rect(screen, black, button,0, 30)

        text = font.render(name, True, white)
        text_button = text.get_rect(center=(button.centerx, button.y+25))
        screen.blit(text, text_button)

        subbutton_1 = subbutton_2 = subbutton_3 = None

        if button.height > 110:
            subbutton_1 = pygame.Rect(button.x+10,button.y+60,button_width-20, button_height)
            pygame.draw.rect(screen, white, subbutton_1,0, 30)
            text_1 = font_2.render(subname_1, True, black)
            text_subbutton_1 = text_1.get_rect(center=subbutton_1.center)
            screen.blit(text_1, text_subbutton_1)

        if button.height > 210:
            subbutton_2 = pygame.Rect(button.x+10,button.y+160,button_width-20, button_height)
            pygame.draw.rect(screen, white, subbutton_2,0, 30)
            text_2 = font_2.render(subname_2, True, black)
            text_subbutton_2 = text_2.get_rect(center=subbutton_2.center)
            screen.blit(text_2, text_subbutton_2)

        if button.height > 310:
            subbutton_3 = pygame.Rect(button.x+10,button.y+260,button_width-20, button_height)
            pygame.draw.rect(screen, white, subbutton_3,0, 30)
            text_3 = font_2.render(subname_3, True, black)
            text_subbutton_3 = text_3.get_rect(center=subbutton_3.center)
            screen.blit(text_3, text_subbutton_3)
        
        return subbutton_1, subbutton_2, subbutton_3

def Math_Game_Uno():
    global i, Math_Game_1
    button_1 = pygame.Rect(width/2-100, height/2-100, 200, 200)
    pygame.draw.rect(screen, black, button_1,0, 30)
    text_math = font_2.render(str(i), True, white)
    text_math_1 = text_math.get_rect(center=button_1.center)
    screen.blit(text_math, text_math_1)
    if i < 500:
        i += 1

def Math_Game_Dos():
    button_2 = pygame.Rect(width/2-100, height/2-100, 200, 200)
    pygame.draw.rect(screen, black, button_2,0, 30)
    text_math_2 = font_2.render("Math Game 2", True, white)
    text_math_1_2 = text_math_2.get_rect(center=button_2.center)
    screen.blit(text_math_2, text_math_1_2)

def Math_Game_Tres():
    button_3 = pygame.Rect(width/2-100, height/2-100, 200, 200)
    pygame.draw.rect(screen, black, button_3,0, 30)
    text_math_3 = font_2.render("Math Game 3", True, white)
    text_math_1_3 = text_math_3.get_rect(center=button_3.center)
    screen.blit(text_math_3, text_math_1_3)

def Eng_Game_Uno():  
    button_4 = pygame.Rect(width/2-100, height/2-100, 200, 200)
    pygame.draw.rect(screen, black, button_4,0, 30)
    text_eng_1 = font_2.render("English Game 1", True, white)
    text_eng_1_1 = text_eng_1.get_rect(center=button_4.center)
    screen.blit(text_eng_1, text_eng_1_1)

def Eng_Game_Dos(): 
    d=d

def Eng_Game_Tres(): 
    e=e

def Chem_Game_Uno(): 
    f=f 

def Chem_Game_Dos(): 
    g=g

def Chem_Game_Tres(): 
    h=h


while running:
    screen.fill(white)

    mouse_x, mouse_y = pygame.mouse.get_pos()

    home_button = pygame.Rect(20,20,50,50)
    pygame.draw.rect(screen, black, home_button,0, 15)
    home_text = font.render("[]", True, white)
    home_text_1 = home_text.get_rect(center=home_button.center)
    screen.blit(home_text, home_text_1)

    if Math_Game_1:
        Math_Game_Uno()
    elif Math_Game_2:
        Math_Game_Dos()
    elif Math_Game_3:
        Math_Game_Tres()
    elif Eng_Game_1:
        Eng_Game_Dos()
    elif Eng_Game_2:
        Eng_Game_Dos()
    elif Eng_Game_3:
        Eng_Game_Tres()
    elif Chem_Game_1:
        Eng_Game_Dos()
    elif Chem_Game_2:
        Eng_Game_Dos()
    elif Chem_Game_3:
        Eng_Game_Tres()
    else:
        subbutton_math_1, subbutton_math_2, subbutton_math_3 = button_dynamics(button_math, 'Math','Game 1','Game 2','Game 3')
        subbutton_eng_1, subbutton_eng_2, subbutton_eng_3 = button_dynamics(button_chem, 'Chemistry','Game 1','Game 2','Game 3')
        subbutton_chem_1, subbutton_chem_2, subbutton_chem_3 = button_dynamics(button_eng,'English','Game 1','Game 2','Game 3')
    
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        if event.type == pygame.MOUSEBUTTONDOWN:
                if home_button.collidepoint(mouse_x, mouse_y):
                    Math_Game_1 = Math_Game_2 = Math_Game_3 = Eng_Game_1 = Eng_Game_2 = Eng_Game_3 = Chem_Game_1 = Chem_Game_2 = Chem_Game_3 = False  
                if subbutton_math_1 and subbutton_math_1.collidepoint(mouse_x, mouse_y):
                    Math_Game_1 = True
                if subbutton_math_2 and subbutton_math_2.collidepoint(mouse_x, mouse_y):
                    Math_Game_2 = True
                if subbutton_math_3 and subbutton_math_3.collidepoint(mouse_x, mouse_y):
                    Math_Game_3 = True
                if subbutton_eng_1 and subbutton_eng_1.collidepoint(mouse_x, mouse_y):
                    Eng_Game_1 = True
                if subbutton_eng_2 and subbutton_eng_2.collidepoint(mouse_x, mouse_y):
                    Eng_Game_2 = True
                if subbutton_eng_3 and subbutton_eng_3.collidepoint(mouse_x, mouse_y):
                    Eng_Game_3 = True
                if subbutton_chem_1 and subbutton_chem_1.collidepoint(mouse_x, mouse_y):
                    Chem_Game_1 = True
                if subbutton_chem_2 and subbutton_chem_2.collidepoint(mouse_x, mouse_y):
                    Chem_Game_2 = True
                if subbutton_chem_3 and subbutton_chem_3.collidepoint(mouse_x, mouse_y):
                    Chem_Game_3 = True
                
    

# Update the display
    pygame.display.flip()

    # Cap the frame rate to 60 fps
    clock.tick(fps)

# Quit Pygame
pygame.quit()
sys.exit()

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83


UnboundLocalError: cannot access local variable 'b' where it is not associated with a value

: 